# 자치구별 코로나 확진자 예측

## [ 서울 열린 데이터 광장 ] 
### 서울특별시 코로나19 자치구별 확진자 발생동향 API
https://data.seoul.go.kr/dataList/OA-20470/S/1/datasetView.do

💡 본 프로그램은 https://github.com/kairess/corona-virus-prediction 의 응용임을 밝힙니다.

In [1]:
# 필요 라이브러리 불러오기
import requests # API를 사용하기 위한 모듈
from bs4 import BeautifulSoup # 텍스트 형태의 HTML과 XML에서 데이터를 추출하기 위한 라이브러리
import pandas as pd # 텍스트 입출력 및 데이터프레임 가공이 가능 한 라이브러리

import plotly.graph_objs as go # 그래프 라이브러리
import plotly.offline as py
from fbprophet import Prophet # 시계열 예측 라이브러리
from fbprophet.plot import plot_plotly, add_changepoints_to_plot

import numpy as np #행렬이나 다차원 배열을 처리할때 사용하는 라이브러리

#### 공공데이터 포털 API

In [2]:
#ServiceKey = 'dypF53MOyB5SvOwH2UtPxnT4vEggzFRRQrSIAV%2BZW3%2FNurg6Mo4Zl5DxsNjqCDHxBVE5hcadyB1%2FxU92iwPtgA%3D%3D'
#url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey="
#api_url = url + ServiceKey

#### 서울데이터열린광장 API

In [2]:
ServiceKey = '74766d63646e61793639476a424f51' # API 인증키
api_url = "http://openAPI.seoul.go.kr:8088/"+ ServiceKey + "/xml/TbCorona19CountStatusJCG/1/5/"
# api_url의 "/xml/TbCorona19CountStatusJCG/1/5/"에서 '5'부분은 가져올 데이터 수. 조절하여 가져올 수 있다.

In [3]:
api_url

'http://openAPI.seoul.go.kr:8088/74766d63646e61793639476a424f51/xml/TbCorona19CountStatusJCG/1/5/'

In [4]:
req = requests.get(api_url) # get method로 HTTP 요청
xml = req.text # url에서 HTTP요청으로 받은 응답을 자동으로 인코딩 방식을 추측하여 그에 맞게 디코딩해준다.
soup = BeautifulSoup(xml, 'html.parser') # BeautifulSoup을 통해 html파싱 후 soup 객체 생성 

In [5]:
soup

<?xml version="1.0" encoding="UTF-8"?>
<tbcorona19countstatusjcg>
<list_total_count>646</list_total_count>
<result>
<code>INFO-000</code>
<message>정상 처리되었습니다</message>
</result>
<row>
<jcg_dt>2021.11.16.00</jcg_dt>
<jongno>2365</jongno>
<jongnoadd>13</jongnoadd>
<junggu>2336</junggu>
<jungguadd>14</jungguadd>
<yongsan>3388</yongsan>
<yongsanadd>10</yongsanadd>
<seongdong>3660</seongdong>
<seongdongadd>38</seongdongadd>
<gwangjin>4371</gwangjin>
<gwangjinadd>23</gwangjinadd>
<ddm>5212</ddm>
<ddmadd>57</ddmadd>
<jungnang>4834</jungnang>
<jungnangadd>37</jungnangadd>
<seongbuk>5359</seongbuk>
<seongbukadd>64</seongbukadd>
<gangbuk>3719</gangbuk>
<gangbukadd>25</gangbukadd>
<dobong>3602</dobong>
<dobongadd>35</dobongadd>
<nowon>5437</nowon>
<nowonadd>26</nowonadd>
<ep>5646</ep>
<epadd>20</epadd>
<sdm>3484</sdm>
<sdmadd>19</sdmadd>
<mapo>4681</mapo>
<mapoadd>29</mapoadd>
<yangcheon>4322</yangcheon>
<yangcheonadd>25</yangcheonadd>
<gangseo>5890</gangseo>
<gangseoadd>41</gangseoadd>
<guro>665

In [6]:
# 조회된 총 데이터 건수 
totalCount = soup.select('list_total_count')
totalCount

[<list_total_count>646</list_total_count>]

### 반복문을 활용한 API 요청

In [7]:
items = soup.select('row')
len(items)

5

In [8]:
DATE = []
JONGNO = []
JONGNO_ADD = []
JUNGGU = []
JUNGGU_ADD = []
YONGSAN = []
YONGSAN_ADD = []
SEONGDONG = []
SEONGDONG_ADD = []
GWANGJIN = []
GWANGJIN_ADD = []
DDM = []
DDM_ADD = []
JUNGNANG = []
JUNGNANG_ADD = []
SEONGBUK = []
SEONGBUK_ADD = []
GANGBUK = []
GANGBUK_ADD = []
DOBONG = []
DOBONG_ADD = []
NOWON = []
NOWON_ADD = []
EP = []
EP_ADD = []
SDM = []
SDM_ADD = []
MAPO = []
MAPO_ADD = []
YANGCHEON = []
YANGCHEON_ADD = []
GURO = []
GURO_ADD = []
GEUMCHEON = []
GEUMCHEON_ADD = []
YDP = []
YDP_ADD = []
DONGJAK = []
DONGJAK_ADD = []
GWANAK = []
GWANAK_ADD = []
SEOCHO = []
SEOCHO_ADD = []
GANGNAM = []
GANGNAM_ADD = []
GANGDONG = []
GANGDONG_ADD = []
ETC = []
ETC_ADD = []

# 서울시 자치구별 확진자 발생동향 
ServiceKey = '74766d63646e61793639476a424f51'
api_url = "http://openAPI.seoul.go.kr:8088/"+ ServiceKey + "/xml/TbCorona19CountStatusJCG/1/200/"

req = requests.get(api_url)
xml = req.text
soup = BeautifulSoup(xml, 'html.parser')   

# 아이템 목록
items = soup.select('row')

for item in items:

    DATE.append(item.select('JCG_DT')[0].text)
    JONGNO.append(item.select('JONGNO')[0].text)
    JONGNO_ADD.append(item.select('JONGNOADD')[0].text)
    JUNGGU.append(item.select('JUNGGU')[0].text)
    JUNGGU_ADD.append(item.select('JUNGGUADD')[0].text)
    YONGSAN.append(item.select('YONGSAN')[0].text)
    YONGSAN_ADD.append(item.select('YONGSANADD')[0].text)
    SEONGDONG.append(item.select('SEONGDONG')[0].text)
    SEONGDONG_ADD.append(item.select('SEONGDONGADD')[0].text)
    GWANGJIN.append(item.select('GWANGJIN')[0].text)
    GWANGJIN_ADD.append(item.select('GWANGJINADD')[0].text)
    DDM.append(item.select('DDM')[0].text)
    DDM_ADD.append(item.select('DDMADD')[0].text)
    JUNGNANG.append(item.select('JUNGNANG')[0].text)
    JUNGNANG_ADD.append(item.select('JUNGNANGADD')[0].text)
    SEONGBUK.append(item.select('SEONGBUK')[0].text)
    SEONGBUK_ADD.append(item.select('SEONGBUKADD')[0].text)
    GANGBUK.append(item.select('GANGBUK')[0].text)
    GANGBUK_ADD.append(item.select('GANGBUKADD')[0].text)
    DOBONG.append(item.select('DOBONG')[0].text)
    DOBONG_ADD.append(item.select('DOBONGADD')[0].text)
    NOWON.append(item.select('NOWON')[0].text)
    NOWON_ADD.append(item.select('NOWONADD')[0].text)
    EP.append(item.select('EP')[0].text)
    EP_ADD.append(item.select('EPADD')[0].text)
    SDM.append(item.select('SDM')[0].text)
    SDM_ADD.append(item.select('SDMADD')[0].text)
    MAPO.append(item.select('MAPO')[0].text)
    MAPO_ADD.append(item.select('MAPOADD')[0].text)
    YANGCHEON.append(item.select('YANGCHEON')[0].text)
    YANGCHEON_ADD.append(item.select('YANGCHEONADD')[0].text)
    GURO.append(item.select('GURO')[0].text)
    GURO_ADD.append(item.select('GUROADD')[0].text)
    GEUMCHEON.append(item.select('GEUMCHEON')[0].text)
    GEUMCHEON_ADD.append(item.select('GEUMCHEONADD')[0].text)
    YDP.append(item.select('YDP')[0].text)
    YDP_ADD.append(item.select('YDPADD')[0].text)
    DONGJAK.append(item.select('DONGJAK')[0].text)
    DONGJAK_ADD.append(item.select('DONGJAKADD')[0].text)
    GWANAK.append(item.select('GWANAK')[0].text)
    GWANAK_ADD.append(item.select('GWANAKADD')[0].text)
    SEOCHO.append(item.select('SEOCHO')[0].text)
    SEOCHO_ADD.append(item.select('SEOCHOADD')[0].text)
    GANGNAM.append(item.select('GANGNAM')[0].text)
    GANGNAM_ADD.append(item.select('GANGNAMADD')[0].text)
    GANGDONG.append(item.select('GANGDONG')[0].text)
    GANGDONG_ADD.append(item.select('GANGDONGADD')[0].text)
    ETC.append(item.select('ETC')[0].text)
    ETC_ADD.append(item.select('ETCADD')[0].text)

In [9]:
DATE[0]

'2021.11.16.00'

In [10]:
len(DATE)

200

### DataFrame 변환

In [11]:
result = {
    'DATE' : DATE,
    'JONGNO' : JONGNO,
    'JONGNO_ADD' : JONGNO_ADD,
    'JUNGGU' : JUNGGU,
    'JUNGGU_ADD' : JUNGGU_ADD,
    'YONGSAN' : YONGSAN,
    'YONGSAN_ADD' : YONGSAN_ADD,
    'SEONGDONG' : SEONGDONG,
    'SEONGDONG_ADD' : SEONGDONG_ADD,
    'GWANGJIN' : GWANGJIN,
    'GWANGJIN_ADD' : GWANGJIN_ADD,
    'DDM' : DDM,
    'DDM_ADD' : DDM_ADD,
    'JUNGNANG' : JUNGNANG,
    'JUNGNANG_ADD' : JUNGNANG_ADD,
    'SEONGBUK' : SEONGBUK,
    'SEONGBUK_ADD' : SEONGBUK_ADD,
    'GANGBUK' : GANGBUK,
    'GANGBUK_ADD' : GANGBUK_ADD,
    'DOBONG' : DOBONG,
    'DOBONG_ADD' : DOBONG_ADD,
    'NOWON' : NOWON,
    'NOWON_ADD' : NOWON_ADD,
    'EP' : EP,
    'EP_ADD' : EP_ADD,
    'SDM' : SDM,
    'SDM_ADD' : SDM_ADD,
    'MAPO' : MAPO,
    'MAPO_ADD' : MAPO_ADD,
    'YANGCHEON' : YANGCHEON,
    'YANGCHEON_ADD' : YANGCHEON_ADD,
    'GURO' : GURO,
    'GURO_ADD' : GURO_ADD,
    'GEUMCHEON' : GEUMCHEON,
    'GEUMCHEON_ADD' : GEUMCHEON_ADD,
    'YDP' : YDP,
    'YDP_ADD' : YDP_ADD,
    'DONGJAK' : DONGJAK,
    'DONGJAK_ADD' : DONGJAK_ADD,
    'GWANAK' : GWANAK,
    'GWANAK_ADD' : GWANAK_ADD,
    'SEOCHO' : SEOCHO,
    'SEOCHO_ADD' : SEOCHO_ADD,
    'GANGNAM' : GANGNAM,
    'GANGNAM_ADD' : GANGNAM_ADD,
    'GANGDONG' : GANGDONG,
    'GANGDONG_ADD' : GANGDONG_ADD,
    'ETC' : ETC,
    'ETC_ADD' : ETC_ADD,
}

data = pd.DataFrame(result)
data

,DATE,JONGNO,JONGNO_ADD,JUNGGU,JUNGGU_ADD,YONGSAN,YONGSAN_ADD,SEONGDONG,SEONGDONG_ADD,GWANGJIN,...,GWANAK,GWANAK_ADD,SEOCHO,SEOCHO_ADD,GANGNAM,GANGNAM_ADD,GANGDONG,GANGDONG_ADD,ETC,ETC_ADD
0,2021.11.16.00,2365,13,2336,14,3388,10,3660,38,4371,...,7057,38,5215,25,8425,37,5500,31,9271,40
1,2021.11.15.00,2352,15,2322,14,3378,18,3622,23,4348,...,7019,32,5190,27,8388,42,5469,78,9232,47
2,2021.11.08.00,2240,6,2208,12,3275,10,3455,16,4208,...,6656,42,4979,26,8133,31,5107,34,8818,51
3,2021.11.07.00,2234,12,2196,9,3265,12,3439,19,4184,...,6614,54,4953,28,8102,34,5073,53,8767,41
4,2021.11.06.00,2222,12,2187,19,3253,15,3420,24,4160,...,6559,57,4925,38,8068,25,5020,40,8712,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2021.04.30.00,680,3,587,0,994,6,1033,5,1228,...,1793,22,1587,12,1939,22,1418,8,3872,15
196,2021.04.29.00,677,3,587,5,988,8,1028,11,1222,...,1778,9,1576,7,1917,19,1410,8,3853,11
197,2021.04.28.00,674,2,582,2,980,5,1017,6,1212,...,1771,14,1569,10,1898,28,1402,13,3841,14
198,2021.04.27.00,672,1,581,1,975,2,1011,3,1202,...,1757,5,1559,3,1870,15,1389,13,3826,5


In [12]:
# csv파일로 변환, 저장
data.to_csv('corona19_regional.csv', index=False)